In [1]:
from args import get_model_args, get_train_args
import constants
from models import get_model
from processes import RepeatedCharsCollapsor, SpacesRemover, ValidCharsKeeper, CharsRemover, CharsNormalizer
from processors import TextProcessor
from tokenizer import get_tokenizer
import torch
from torch.nn import Module
from torch import Tensor, BoolTensor
from utils import load_state
from predict import get_predictor
import sys

In [2]:
processes = [
    RepeatedCharsCollapsor(2),
    CharsRemover(constants.ARABIC_HARAKAT),
    CharsNormalizer(constants.NORMLIZER_MAPPER),
    ValidCharsKeeper(constants.VALID_CHARS),
    SpacesRemover()
]
processor = TextProcessor(processes)

In [3]:
sys.argv=['']
device = 'cuda'
max_len = 200
checkpoint_path = 'outdir/checkpoint.pt' # checkpoint path
args = get_train_args()
args.tokenizer_path = 'outdir/tokenizer.json' # tokenizer path
args.hidden_size = 256
args.n_layers = 3
args.model = 'seq2seqrnn' # transformer, seq2seqrnn, basicrnn
tokenizer = get_tokenizer(args)
model = get_model(
    args, voc_size=tokenizer.vocab_size, rank=0, pad_idx=tokenizer.special_tokens.pad_id
    )
model.load_state_dict(load_state(checkpoint_path)[0])
_ = model.to(device).eval()

predictor = get_predictor(
    args=args,
    model=model,
    tokenizer=tokenizer,
    max_len=max_len,
    processor=processor,
    device=device
)

tokenizer outdir/tokenizer.json loadded


In [ ]:
text = 'ويختلف اللون باختلاف ابليئةالتتعيش فيها'
predictor.predict(text)